In [1]:
import numpy as np
import matplotlib.pyplot as plt
import napari
import pandas as pd
from anndata import AnnData, read_h5ad
import scanpy as sc
import matplotlib as mpl


In [2]:
%matplotlib qt
#%matplotlib inline

In [3]:
"""plot style"""
linewidth = 4
mpl.rc('axes', linewidth=linewidth)
mpl.rc('font', family='Arial')
fontsize = 24


colors = {'no_inj': [0.8, 0.8, 0.8],
         'mock': [0.4, 0.4, 0.4],
         'e.coli': [0, 0.4, 0],
         'complete': [0, 0.8, 0]}

def style_axes(ax, fontsize=24):
    plt.minorticks_off()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    plt.tight_layout()
    
    return ax


In [4]:
def bin_aps(aps, bins):
    _counts, bins = np.histogram(aps, bins)
    bins = bins[1:]
    binned_aps = np.zeros_like(aps)
    for i in range(len(binned_aps)):
        binned_aps[i] = get_ap_bin(aps[i], bins)
    
    return binned_aps
    

def get_ap_bin(this_ap, bins):
    this_bin = np.where(np.abs(this_ap - bins) == np.nanmin(np.abs(this_ap - bins)))[0][0]

    return this_bin


def line_dist(gene, adata, key, n_bootstraps=10):
    this_gene_index = np.where(adata.var_names == gene)[0][0]
    ap = np.unique(adata.obs[key])
    X = adata.layers['log1p']
    #X = adata.X

    # arrays for storing output
    mean_expression = np.zeros(len(np.unique(ap)))
    std_dev = np.zeros(len(np.unique(ap)))
    bootstrapped_expression = np.zeros((n_bootstraps, len(np.unique(ap))))
    scrambled_expression = np.zeros((n_bootstraps, len(np.unique(ap))))
    scrambled_std_dev = np.zeros(len(np.unique(ap)))
    n_cells = X.shape[0]
   
    
    for i in range(len(np.unique(ap))):
        these_cell_indices = adata.obs[key] == ap[i]
        mean_expression[i] = np.mean(X[these_cell_indices, this_gene_index])
        #mean_expression[i] = np.mean(X[these_cell_indices])

        std_dev[i] = np.std(X[these_cell_indices, this_gene_index])
    
    for n in range(n_bootstraps):
        scrambled_ap = np.random.choice(adata.obs[key], size=n_cells, replace=False)
        for i in range(len(np.unique(ap))):
            these_cell_indices = np.random.choice(np.where(adata.obs[key] == ap[i])[0], size=np.sum(adata.obs[key] == ap[i]))
            bootstrapped_expression[n, i] = np.mean(adata.layers['log1p'][these_cell_indices, this_gene_index])
            these_cell_indices = scrambled_ap == ap[i]
            scrambled_expression[n, i] = np.mean(adata.layers['log1p'][these_cell_indices, this_gene_index])
            scrambled_std_dev[i] = np.std(adata.layers['log1p'][these_cell_indices, this_gene_index])  
    #std_dev = np.std(bootstrapped_expression, axis=0)      
    
    #scrambled_mean_expression = np.mean(scrambled_expression, axis=0)
    #scrambled_std_dev = np.std(scrambled_expression, axis=0)
    rest_sel = adata.obs[key] > 0.3
    scrambled_mean_expression = np.mean(X[rest_sel, this_gene_index]) * np.ones_like(ap)
    scrambled_std_dev = np.std(X[rest_sel, this_gene_index]) * np.ones_like(ap) 

    # scrambled_mean_expression = np.mean(X[:, this_gene_index]) * np.ones_like(ap)
    # scrambled_std_dev = np.std(X[:, this_gene_index]) * np.ones_like(ap) 

    return mean_expression, std_dev, scrambled_mean_expression, scrambled_std_dev, ap

                            

                


## Load and preprocess the data

In [5]:
file_path = r'/media/brandon/Data2/Brandon/fly_immune/Flysta3d/L3_b_count_normal_stereoseq.h5ad'
adata = read_h5ad(file_path)

In [12]:
"""create AP bins"""
n_y_bins = 5
all_ys = np.array(adata.obs['new_y'])
y_bins = np.linspace(np.min(all_ys), np.max(all_ys), n_y_bins)
print(f'The anterior-posterior axis goes from {np.min(all_ys)} to {np.max(all_ys)}')

The anterior-posterior axis goes from -178.66514999999998 to 193.73085


In [146]:
"""filter reads to 5% detection"""
detection_percent = np.sum(adata.layers['raw_counts'] > 0, axis=0) / len(adata.layers['raw_counts'])
detection_sel = detection_percent > 0.05
adata = adata[:, detection_sel]

"""get selection of just fat body cells"""
annotation = np.array(adata.obs['annotation'])
fb_sel = (annotation == 'fat body')
#fb_sel = (annotation == 'carcass')
#fb_sel = (annotation == 'muscle')

adata = adata[fb_sel]

"""replace the log1p-normalized dataset with analytic pearson residuals. store log1p as a new layer"""
adata.layers['log1p'] = adata.X
adata.X = sc.experimental.pp.normalize_pearson_residuals(adata, layer='raw_counts', theta=50, inplace=False)['X']

/tmp/ipykernel_4028137/3732076482.py:15: ImplicitModificationWarning: Setting element `.layers['log1p']` of view, initializing view as actual.
  adata.layers['log1p'] = adata.X


In [147]:
"""bin the AP axis"""
ys = np.array(adata.obs['new_y'])
binned_ys = bin_aps(ys, y_bins)
ap_labels = ['0-25%', '25-50%', '50-75%', '75-100%']
adata.obs['anterior_posterior_position'] = [ap_labels[int(b)] for b in binned_ys]
adata.obs['anterior_posterior_position'] = adata.obs['anterior_posterior_position'].astype('category')

In [148]:
adata

AnnData object with n_obs × n_vars = 6628 × 6505
    obs: 'slice_ID', 'raw_x', 'raw_y', 'new_x', 'new_y', 'new_z', 'annotation', 'anterior_posterior_position'
    uns: 'annotation_colors'
    obsm: 'X_umap', 'spatial'
    layers: 'raw_counts', 'log1p'

## Clustering and UMAP

In [149]:
"""run the pre-clustering and clustering routines"""
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, n_iterations=2, resolution=0.04, flavor='igraph')

## Plot UMAPs colored by both clusters and AP position

In [66]:
"""colored by cluster"""
sc.pl.umap(adata, color='leiden', legend_fontsize=0.75 * fontsize)
plt.xlabel('')
plt.ylabel('')
plt.title('leiden subclusters', fontsize=fontsize)
ax = style_axes(plt.gca())


In [26]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/L3_fat_body_subclusters_umap.pdf')

In [151]:
"""colored by AP"""
sc.pl.umap(adata, color='anterior_posterior_position', legend_fontsize=0.75 * fontsize, palette='Dark2')
plt.xlabel('')
plt.ylabel('')
plt.title('anterior-posterior position', fontsize=fontsize)
ax = style_axes(plt.gca())

In [33]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/L3_fat_body_ap_umap.pdf')

In [150]:
sc.pl.umap(adata, color='Act5C', legend_fontsize=0.75 * fontsize)
plt.xlabel('')
plt.ylabel('')
plt.title('Act5C', fontsize=fontsize)
ax = style_axes(plt.gca())

In [153]:
sc.pl.scatter(adata, 'new_y', 'TotA')

## View clusters spatially within the larva

In [34]:
# get colormap to match UMAP clusters. For now, the default mpl color cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# extract the points of each cell and swap x and z coordinates so to comply with naparis zyx convention.
points = adata.obsm['spatial']
points[:, [0,2]] = points[:, [2,0]]

# view each cluster's points at a time. convert to a DataFrame first for convenience
points_df = pd.DataFrame({'z': points[:,0], 'y': points[:,1], 'x':points[:,2], 'cluster':adata.obs['leiden'].values})
viewer = napari.Viewer()
for i, p in enumerate(points_df.cluster.unique()):
    these_points = points_df[points_df.cluster == p].get(['z', 'y', 'x']).values
    viewer.add_points(these_points, size=2, face_color=colors[i], name=f'subcluster {i}')

# widen the z coordinate for clarity
for layer in viewer.layers:
   layer.scale = (5, 1, 1)

viewer.dims.ndisplay = 3

In [76]:
viewer

Viewer(camera=Camera(center=(29.75, 2.8964249999999936, -3.783050000000003), zoom=3.2259191074211735, angles=(-77.29686970104825, -48.29293429423288, -80.65099051939806), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(3.528132420555469, -62.851850803801184, -17.322788454203238), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=3, last_used=0, range=((3.5, 61.0, 5.0), (-178.21475, 185.0076, 1.0), (-72.07325, 65.50715, 1.0)), current_step=(2, 104, 51), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Points layer 'these_points' at 0x7fc834465be0>, <Points layer 'these_points [1]' at 0x7fc8345128d0>, <Points layer 'these_points [2]' at 0x7fc834512f00>, <Points layer 'these_points [3]' at 0x7fc83413ac00>, <Points layer 'these_points [4]' at 0x7fc82d988aa0>, <Points layer 'these_points [5]' at 0x7fc834374290>], help='use <5> for transform, use <2> for add po

## Identify marker genes that separate subclusters
While we used pearson residuals to identify subclusters, we use regular log1p normalized counts for differential expression analysis

In [155]:
"""identify and filter marker genes"""
sc.tl.rank_genes_groups(adata, groupby='leiden', method="wilcoxon", corr_method='bonferroni', layer='log1p', pts=True)


In [156]:
sc.tl.filter_rank_genes_groups(adata, groupby='leiden', min_fold_change=1, min_in_group_fraction=0, max_out_group_fraction=1)

In [38]:
sc.pl.rank_genes_groups_heatmap(
    adata, groupby='leiden', key='rank_genes_groups_filtered', n_genes=5, vmin=0, vmax=10, show_gene_labels=True, min_logfoldchange=1)

In [41]:
sc.pl.rank_genes_groups_matrixplot(
    adata, groupby='leiden', key='rank_genes_groups_filtered', n_genes=5, vmin=0, vmax=10,  min_logfoldchange=1, cmap='magma')

## Volcano plot of differentially-expressed genes

In [108]:
lfc = adata.uns['rank_genes_groups']['logfoldchanges']['2']
pvals = adata.uns['rank_genes_groups']['pvals']['2']
genes2 = adata.uns['rank_genes_groups']['names']['2']
#pvals[pvals == 0] = np.min(pvals[pvals > 0] / 10)
#pvals_adj = adata.uns['rank_genes_groups']['pvals_adj']['2']

plt.figure()
# not significant
selection = (pvals_adj > 0.05) | np.abs(lfc < 2.5)
#plt.plot(lfc[selection], -np.log10(pvals[selection]), 'ko', markersize=3, alpha=0.1)
plt.plot(lfc[~selection], -np.log10(pvals[~selection]), 'go', markersize=3, alpha=0.5)

# DptA
this_gene = 'DptA'
plt.plot(lfc[this_gene==genes2], -np.log10(pvals[this_gene==genes2]), 'mo', markersize=8, alpha=0.5)
this_gene = 'regucalcin'
plt.plot(lfc[this_gene==genes2], -np.log10(pvals[this_gene==genes2]), 'mo', markersize=8, alpha=0.5)
this_gene = 'TotA'
plt.plot(lfc[this_gene==genes2], -np.log10(pvals[this_gene==genes2]), 'mo', markersize=8, alpha=0.5)

plt.xlabel('log2 fold change', fontsize=fontsize)
plt.ylabel('-log10 p-value', fontsize=fontsize)
ax = style_axes(plt.gca())

In [98]:
len(genes2)

6505

In [97]:
len(genes2[~selection])

1349

In [100]:
for g in genes2[~selection]:
    print(g)

regucalcin
CG42834
CG14332
CG33333
LysS
CG9682
lncRNA:CR33938
nw
lncRNA:cherub
E(spl)m8-HLH
E(spl)m4-BFM
CG11300
l(3)neo38
Brd
CG34215
28SrRNA-Psi:CR45855
CG13905
Ppr-Y
7SLRNA:CR42652
kl-5
CG15293
7SLRNA:CR32864
ORY
CG32073


In [87]:
genes2[lfc == np.min(lfc)]

array(['FASN3'], dtype=object)

In [80]:
np.where(adata.var_names == 'TotA')

(array([4958]),)

In [64]:
lfc_rec['0']

array([ 1.7242341,  4.1820145,  1.915775 , ..., -3.4468439, -2.9429138,
       -4.537663 ], dtype=float32)

In [61]:
np.array(lfc_rec[i], dtype='float')

TypeError: Cannot cast array data from dtype((numpy.record, [('0', '<f4'), ('1', '<f4'), ('2', '<f4'), ('3', '<f4'), ('4', '<f4'), ('5', '<f4')])) to dtype('float64').

In [50]:
sc.pl.scatter(adata, x='logfoldchanges', y='pvals')

ValueError: `x`, `y`, and potential `color` inputs must all come from either `.obs` or `.var`

In [43]:
adata.uns['rank_genes_groups']

{'params': {'groupby': 'leiden',
  'reference': 'rest',
  'method': 'wilcoxon',
  'use_raw': False,
  'layer': 'log1p',
  'corr_method': 'bonferroni'},
 'pts':                             0         1         2         3         4  \
 geneID                                                                  
 128up                0.078901  0.151344  0.139831  0.061122  0.105263   
 14-3-3epsilon        0.619681  0.702082  0.870056  0.560176  0.852632   
 14-3-3zeta           0.585106  0.587164  0.820621  0.555135  0.800000   
 18SrRNA-Psi:CR41602  0.667553  0.770598  0.368644  0.344675  0.578947   
 26-29-p              0.271277  0.317433  0.401130  0.287965  0.463158   
 ...                       ...       ...       ...       ...       ...   
 zetaTry              0.000887  0.163920  0.108757  0.205419  0.168421   
 zf30C                0.049645  0.066349  0.187853  0.041588  0.094737   
 zip                  0.365248  0.301821  0.467514  0.268431  0.547368   
 zormin               0.142

In [ ]:
sc.pl.scatter(adata, layersa

## Plot AP-distributions of identified genes

In [186]:
"""create finer ap bins for plotting"""
n_y_bins = 16
fine_y_bins = np.linspace(np.min(all_ys), np.max(all_ys), n_y_bins)
binned_ys = bin_aps(ys, fine_y_bins)
fraction_ap = np.linspace(0, 1, n_y_bins - 1)
adata.obs['anterior_posterior_position_fine'] = [fraction_ap[int(b)] for b in binned_ys]

In [305]:
"""just try looking at top 5 genes by pvalue"""
anterior_cluster = 3#2
anterior_genes = []
n_top_genes = 15
for i in range(n_top_genes):
    anterior_genes.append(adata.uns['rank_genes_groups']['names'][i][anterior_cluster])

In [109]:
anterior_genes = genes2[~selection]

In [110]:
anterior_genes[-1] = 'DptA'

In [122]:
anterior_genes[2] = 'Act5C'

In [158]:
anterior_genes= filtered_genes
anterior_genes.append('Act5C')


In [188]:
anterior_genes = ['Act42A',
 'Adf1',
 'Adh',
 'Ank2',
 'Aos1',
 'Appl',
 'B52']

In [191]:
fig, axes = plt.subplots(5, 3)
counter = 0
#plot_ap = np.arange(2, n_y_bins - 3)
plot_ap = np.arange(0, n_y_bins - 1)

for i in range(5):
    for j in range(3):
        mean_expression, std_dev, scrambled_mean_expression, scrambled_std, ap = line_dist(
            anterior_genes[counter], adata, key='anterior_posterior_position_fine', n_bootstraps=2)
        axes[i, j].errorbar(ap[plot_ap], mean_expression[plot_ap], std_dev[plot_ap], marker='o', markerfacecolor='g', 
                     markeredgecolor='k', markeredgewidth=1, markersize=12, 
                     capsize=4, capthick=4, linewidth=2, color='k', ecolor='k', barsabove=True, zorder=100)
        
        l = scrambled_mean_expression - scrambled_std
        u = scrambled_mean_expression + scrambled_std
        axes[i, j].fill_between(ap[plot_ap], l[plot_ap], u[plot_ap], facecolor='k', alpha=0.2)
        axes[i, j].plot(ap[plot_ap], scrambled_mean_expression[plot_ap], '--', color=[0.5, 0.5, 0.5], linewidth=4)
        axes[i, j].set_title(anterior_genes[counter])
        #axes[i, j].set_ylim([0, 5])
        counter += 1

IndexError: list index out of range

In [126]:
this_gene_index = np.where(adata.var_names == 'DptA')[0][0]
intens = adata.layers['log1p'][:, this_gene_index]

In [128]:
np.sum(intens == 0) / len(intens)

0.8866928183464092

In [129]:
np.mean(intens)

array(0.10836635, dtype=float32)

In [130]:
np.max(intens)

array(2.7274914, dtype=float32)

In [131]:
np.std(intens)

0.34334874

In [135]:
plt.figure()
plt.hist(np.exp(intens) - 1, bins=np.arange(12))
plt.yscale('log')

In [160]:
mean_expression, std_dev, scrambled_mean_expression, scrambled_std, ap = line_dist(
            anterior_genes[0], adata, key='anterior_posterior_position_fine', n_bootstraps=100)

In [154]:
adata.obs[].values

array([0.42857143, 0.42857143, 0.42857143, ..., 0.57142857, 0.57142857,
       0.57142857])

In [177]:
adata.uns['rank_genes_groups']['names']

rec.array([('CG42500', 'CG34168', 'regucalcin', 'CG12057', 'CG14300', 'Jon65Aiv'),
           ('Sgs8', 'CG17376', 'TotA', 'Jon99Ciii', 'Peritrophin-15a', 'epsilonTry'),
           ('Lcp4', 'Mst84Db', 'Hsp23', 'Jon99Cii', 'CG34282', 'deltaTry'),
           ...,
           ('Jon65Aiv', 'CG31698', 'asRNA:CR11538', 'Lcp4', 'CG16704', 'CG5773'),
           ('CG14302', 'CG15404', 'CG32073', 'CG42500', 'CG34166', 'CG16713'),
           ('Drsl2', 'CG7606', 'Lsp1alpha', 'TotA', 'Obp99b', 'CG32073')],
          dtype=[('0', 'O'), ('1', 'O'), ('2', 'O'), ('3', 'O'), ('4', 'O'), ('5', 'O')])

In [ ]:
fc_arr = np.zeros((len(ranked_genes), len(ranked_genes[0])))
for i in range(len(fc_arr)):
    fc_arr[i] = adata.uns['rank_genes_groups']['logfoldchange

In [186]:
same_params

True

In [187]:
"logfoldchanges" in adata.uns[key]

True

In [93]:
fig, ax = plt.subplots()
counter = 0
ub_genes = ['FASN3']#['Act5C', 'betaTub56D', 'alphaTub84B', 'FASN3']
for counter in range(len(ub_genes)):
        mean_expression, std_dev, scrambled_mean_expression, scrambled_std, ap = line_dist(
            ub_genes[counter], adata, key='anterior_posterior_position_fine', n_bootstraps=100)
        norm = 1#mean_expression[5]
        ax.errorbar(ap, mean_expression / norm, std_dev/ norm, marker='o', markerfacecolor='g', 
                     markeredgecolor='k', markeredgewidth=1, markersize=12, 
                     capsize=4, capthick=4, linewidth=2, color='k', ecolor='k', barsabove=True, zorder=100)
        
        

In [217]:
np.where(adata.var_names == 'Act5c')[0][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [254]:
plt.figure()
plt.hist(adata.obs['new_y'])

(array([ 646., 1477.,  810.,  777.,  886.,  761.,  498.,  333.,  252.,
          59.]),
 array([-178.66515, -141.42555, -104.18595,  -66.94635,  -29.70675,
           7.53285,   44.77245,   82.01205,  119.25165,  156.49125,
         193.73085]),
 <BarContainer object of 10 artists>)

In [270]:
np.min(ys)

-156.26985

In [320]:
plt.close('all')

# Pseudo bulk analysis

### Load + preprocess the data and find clusters as usual. We will use these clusters as grouping for the pseudobulking

In [9]:
file_path = r'/media/brandon/Data2/Brandon/fly_immune/Flysta3d/L3_b_count_normal_stereoseq.h5ad'
adata = read_h5ad(file_path)

"""filter reads to 5% detection"""
detection_percent = np.sum(adata.layers['raw_counts'] > 0, axis=0) / len(adata.layers['raw_counts'])
detection_sel = detection_percent > 0.05
adata = adata[:, detection_sel]

"""get selection of just fat body cells"""
annotation = np.array(adata.obs['annotation'])
fb_sel = (annotation == 'fat body')
#fb_sel = (annotation == 'carcass')
#fb_sel = (annotation == 'muscle')

adata = adata[fb_sel]


"""replace the log1p-normalized dataset with analytic pearson residuals. store log1p as a new layer"""
adata.layers['log1p'] = adata.X
adata.X = sc.experimental.pp.normalize_pearson_residuals(adata, layer='raw_counts', theta=50, inplace=False)['X']

"""run the pre-clustering and clustering routines"""
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, n_iterations=2, resolution=0.04, flavor='igraph')

/tmp/ipykernel_477553/3894648728.py:19: ImplicitModificationWarning: Setting element `.layers['log1p']` of view, initializing view as actual.
  adata.layers['log1p'] = adata.X
/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [176]:
counts = adata.copy()
counts.X = counts.layers['raw_counts']
sc.pp.normalize_total(counts)

In [128]:
counts

AnnData object with n_obs × n_vars = 6628 × 6505
    obs: 'slice_ID', 'raw_x', 'raw_y', 'new_x', 'new_y', 'new_z', 'annotation', 'leiden'
    uns: 'annotation_colors', 'pca', 'neighbors', 'umap', 'leiden'
    obsm: 'X_umap', 'spatial', 'X_pca'
    varm: 'PCs'
    layers: 'raw_counts', 'log1p'
    obsp: 'distances', 'connectivities'

In [177]:
n_bootstraps = 100
clusters = counts.obs['leiden'].unique()
grouped_counts = np.zeros((n_bootstraps, len(clusters), counts.shape[1]))

for n in range(n_bootstraps):
    for i in range(len(clusters)):
        these_counts = counts.X[counts.obs['leiden'] == clusters[i]]
        random_ids = np.random.choice(np.arange(len(these_counts)), size=len(these_counts))
        grouped_counts[n, i] = np.sum(these_counts[random_ids], axis=0)

actin_index = np.where(counts.var_names == 'Act5C')[0][0]
#grouped_counts = grouped_counts / np.expand_dims(grouped_counts[:, :, actin_index], axis=2)

log1p_counts = np.log(grouped_counts + 1)

In [178]:
mean_bulk = np.mean(log1p_counts, axis=0)
std_bulk = np.std(log1p_counts, axis=0)

In [179]:
mean_bulk_anterior = mean_bulk[2]
std_bulk_anterior = std_bulk[2]

mean_bulk_rest = np.mean(mean_bulk[[0, 1, 3, 4, 5], :], axis=0)
std_bulk_rest = np.sqrt(np.mean(std_bulk[[0, 1, 3, 4, 5], :], axis=0))

In [132]:
"""filter by standard deviation"""
std_multiplier = 2
good_ids = np.where(mean_bulk_anterior - std_multiplier * std_bulk_anterior > mean_bulk_rest + std_multiplier * std_bulk_rest)[0]
good_genes = counts.var_names[good_ids].tolist()

In [180]:
"""filter by standard deviation using Act5C as threshold"""
differences = mean_bulk_anterior - std_bulk_anterior - mean_bulk_rest -  std_bulk_rest
good_ids = np.where(differences > differences[actin_index])[0]
good_genes = counts.var_names[good_ids].tolist()

In [182]:
differences[actin_index]

0.33382545461977775

In [183]:
"""futher filter by cross referencing a list of genes expressed in the larval fat body by bulk RNA seq"""
bulk_df = pd.read_excel(r'/home/brandon/Downloads/GSE95800_GEO_upload_processed_data.xlsx')
bulk_df
bulk_genes = bulk_df.Gene.to_list()

filtered_genes = []
filtered_ids = []
for i, g in enumerate(good_genes):
    if g in bulk_genes:
        filtered_genes.append(g)
        filtered_ids.append(good_ids[i])

filtered_ids = np.array(filtered_ids)


In [184]:
plt.figure()
counter = 0
for i in range(len(filtered_ids)):
    if i == 0:
        label = 'anterior'
    else:
        label= '_nolabel_'
    plt.errorbar(counter, mean_bulk_anterior[filtered_ids[i]], std_bulk_anterior[filtered_ids[i]],marker='o', markerfacecolor='g', 
                     markeredgecolor='k', markeredgewidth=1, markersize=12, 
                     capsize=4, capthick=4, linewidth=2, color='k', ecolor='k', barsabove=True, zorder=100, label=label) 
    if i == 0:
        label = 'rest'
    else:
        label= '_nolabel_'
    counter += 1
    plt.errorbar(counter, mean_bulk_rest[filtered_ids[i]], std_bulk_rest[filtered_ids[i]],marker='o', markerfacecolor='m', 
                 markeredgecolor='k', markeredgewidth=1, markersize=12, 
                 capsize=4, capthick=4, linewidth=2, color='k', ecolor='k', barsabove=True, zorder=100, label=label) 
    counter += 6
    
plt.xticks(np.arange(0, 7 * len(filtered_ids), 7), labels=filtered_genes, rotation='vertical')
plt.ylabel('gene expresion, $log(x/x_{actin} + 1$)')
plt.legend()

## Pseudobulk using simple mean and std. dev

In [201]:
clusters = counts.obs['leiden'].unique()
mean_bulk = np.zeros((len(clusters), counts.shape[1]))
std_bulk = np.zeros((len(clusters), counts.shape[1]))
for i in range(len(clusters)):
    these_counts = counts.X[counts.obs['leiden'] == clusters[i]]
    mean_bulk[i] = np.mean(np.log2(these_counts+1), axis=0)
    std_bulk[i] = np.std(np.log2(these_counts+1), axis=0)

actin_index = np.where(counts.var_names == 'Act5C')[0][0]

log1p_counts = np.log(grouped_counts + 1)

In [202]:
mean_bulk_anterior = mean_bulk[2]
std_bulk_anterior = std_bulk[2]

mean_bulk_rest = np.mean(mean_bulk[[0, 1, 3, 4, 5], :], axis=0)
std_bulk_rest = np.sqrt(np.mean(std_bulk[[0, 1, 3, 4, 5], :], axis=0))

In [207]:
"""filter by standard deviation using Act5C as threshold"""
differences = (mean_bulk_anterior - std_bulk_anterior - mean_bulk_rest -  std_bulk_rest) / mean_bulk_anterior
good_ids = np.where(differences > differences[actin_index])[0]
good_genes = counts.var_names[good_ids].tolist()

In [208]:
good_genes

['CG14302',
 'CG7606',
 'Fbp1',
 'Hsp23',
 'RpL39',
 'RpLP2',
 'TotA',
 'eEF1alpha1',
 'lncRNA:CR34335',
 'lncRNA:CR40469',
 'regucalcin']

In [209]:
"""futher filter by cross referencing a list of genes expressed in the larval fat body by bulk RNA seq"""
bulk_df = pd.read_excel(r'/home/brandon/Downloads/GSE95800_GEO_upload_processed_data.xlsx')
bulk_df
bulk_genes = bulk_df.Gene.to_list()

filtered_genes = []
filtered_ids = []
for i, g in enumerate(good_genes):
    if g in bulk_genes:
        filtered_genes.append(g)
        filtered_ids.append(good_ids[i])

filtered_ids = np.array(filtered_ids)


In [210]:
filtered_genes

['CG14302', 'CG7606', 'Fbp1', 'Hsp23', 'RpL39', 'RpLP2', 'TotA', 'regucalcin']

In [215]:
filtered_genes[0] = 'Act5C'

In [211]:
"""create finer ap bins for plotting"""
n_y_bins = 16
fine_y_bins = np.linspace(np.min(all_ys), np.max(all_ys), n_y_bins)
binned_ys = bin_aps(ys, fine_y_bins)
fraction_ap = np.linspace(0, 1, n_y_bins - 1)
adata.obs['anterior_posterior_position_fine'] = [fraction_ap[int(b)] for b in binned_ys]

In [220]:
fig, axes = plt.subplots(4, 2)
counter = 0
#plot_ap = np.arange(2, n_y_bins - 3)
plot_ap = np.arange(0, n_y_bins - 1)

for i in range(4):
    for j in range(2):
        mean_expression, std_dev, scrambled_mean_expression, scrambled_std, ap = line_dist(
            filtered_genes[counter], adata, key='anterior_posterior_position_fine', n_bootstraps=1)
        axes[i, j].errorbar(ap[plot_ap], mean_expression[plot_ap], std_dev[plot_ap], marker='o', markerfacecolor='g', 
                     markeredgecolor='k', markeredgewidth=1, markersize=12, 
                     capsize=4, capthick=4, linewidth=2, color='k', ecolor='k', barsabove=True, zorder=100)
        
        l = scrambled_mean_expression - scrambled_std
        u = scrambled_mean_expression + scrambled_std
        #axes[i, j].fill_between(ap[plot_ap], l[plot_ap], u[plot_ap], facecolor='k', alpha=0.2)
        axes[i, j].plot(ap[plot_ap], scrambled_mean_expression[plot_ap], '--', color=[0.5, 0.5, 0.5], linewidth=4)
        axes[i, j].set_title(filtered_genes[counter])
        axes[i, j].set_ylim([0, 5])
        counter += 1

## Normalizing by coarse-grained actin

In [6]:
counts = adata.copy()
counts.X = counts.layers['raw_counts']
sc.pp.normalize_total(counts)
actin_index = np.where(counts.var_names == 'Act5C')[0][0]


In [7]:
counts

AnnData object with n_obs × n_vars = 43310 × 16326
    obs: 'slice_ID', 'raw_x', 'raw_y', 'new_x', 'new_y', 'new_z', 'annotation'
    uns: 'annotation_colors'
    obsm: 'X_umap', 'spatial'
    layers: 'raw_counts'

In [8]:
"""create spatial bins"""
n_y_bins = 30
n_x_bins = 15
n_z_bins = 8
fine_y_bins = np.linspace(np.min(counts.obs['new_y']), np.max(counts.obs['new_y']), n_y_bins)
fine_z_bins = np.linspace(np.min(counts.obs['new_z']), np.max(counts.obs['new_z']), n_z_bins)
fine_x_bins = np.linspace(np.min(counts.obs['new_x']), np.max(counts.obs['new_x']), n_x_bins)

ys = np.array(counts.obs['new_y'])
zs = np.array(counts.obs['new_z'])
xs = np.array(counts.obs['new_x'])

counts.obs['binned_y'] = fine_y_bins[bin_aps(ys, fine_y_bins).astype('int')]
counts.obs['binned_z'] = fine_z_bins[bin_aps(zs, fine_z_bins).astype('int')]
counts.obs['binned_x'] = fine_x_bins[bin_aps(xs, fine_x_bins).astype('int')]

In [9]:
actin_field = np.zeros((n_z_bins, n_y_bins, n_x_bins))
actin_counts = np.array(counts.X[:, actin_index])
for k in range(n_z_bins - 1):
    for j in range(n_y_bins - 1):
        for i in range(n_x_bins - 1):
            actin_sel = (counts.obs.binned_z == fine_z_bins[k]) * (counts.obs.binned_y == fine_y_bins[j]) * (counts.obs.binned_x == fine_x_bins[i])
            if np.sum(actin_sel) > 0:
                actin_field[k, j, i] = np.mean(actin_counts[actin_sel])
            

In [63]:
viewer = napari.view_image(actin_field)

In [42]:
fine_y_bins

array([-178.21475   , -154.06659333, -129.91843667, -105.77028   ,
        -81.62212333,  -57.47396667,  -33.32581   ,   -9.17765333,
         14.97050333,   39.11866   ,   63.26681667,   87.41497333,
        111.56313   ,  135.71128667,  159.85944333,  184.0076    ])

In [43]:
fine_x_bins

array([-72.07325, -62.96789, -53.86253, -44.75717, -35.65181, -26.54645,
       -17.44109,  -8.33573,   0.76963,   9.87499,  18.98035,  28.08571,
        37.19107,  46.29643,  55.40179,  64.50715])

In [52]:
fine_z_bins

array([ 0.7,  1.4,  2.1,  2.8,  3.5,  4.2,  4.9,  5.6,  6.3,  7. ,  7.7,
        8.4,  9.1,  9.8, 10.5, 11.2])

In [53]:
counts.obs['new_z'].unique()

array([ 0.7,  1.4,  2.1,  2.8,  3.5,  4.2,  4.9,  5.6,  6.3,  7. ,  7.7,
        8.4,  9.1,  9.8, 10.5, 11.2])

In [65]:
from scipy.optimize import least_squares

In [85]:
def poly_field(shape, params):
    z, y, x = np.indices(shape)
    c0, c1, c2, c3, c4, c5, c6, c7, c8, c9 = params

    field = c0 + c1 * x + c2 * y + c3 * z + c4 * x * y + c5 * y * z + c6 * x * z + c7 * x ** 2 + c8 * y ** 2 + c9 * z ** 2
    field[field < 0] = 0
    
    return field

def get_inital_param_guesses(data):
    params = np.zeros(10)
    params[0] = np.min(data[data > 0])
    params[4] = 1
    params[3] = 1
    return np.ones(10)


def fit_poly_field(data):
    params = get_inital_param_guesses(data)
    
    def error_function(p): return np.ravel(
    poly_field(data.shape, p) - data)
        
    result = least_squares(error_function, params, method='trf')

    return result
        

In [86]:
res = fit_poly_field(actin_field)

In [87]:
fit = poly_field(actin_field.shape, res.x)

In [88]:
viewer.add_image(fit)

<Image layer 'fit [1]' at 0x7f75ccbecda0>

In [80]:
res.x

array([1.49373042e+00, 1.23573341e-18, 3.36605181e-11, 4.03302453e-18,
       1.82273575e-03, 5.79308514e-15, 5.20858607e-17, 5.21081849e-17,
       1.77094652e-14, 1.83644513e-21])

In [23]:
from scipy.interpolate import interpn

In [18]:
points = np.indices(actin_field.shape).astype('float')
for i in range(len(points)):
    points[i] = points[i] / points[i].shape[i]

In [30]:
z = np.linspace(0, 1, actin_field.shape[0])
y = np.linspace(0, 1, actin_field.shape[1])
x = np.linspace(0, 1, actin_field.shape[2])
points = (z, y, x)

In [41]:
xi = np.array([0.5, 0.5, 0.5])

In [76]:
nz = 100
ny = 300
nx = 200
z = np.linspace(0, 1, nz)
y = np.linspace(0, 1, ny)
x = np.linspace(0, 1, nx)
xi = np.zeros((nz * ny * nx, 3))
counter = 0
for k in range(nz):
    for j in range(ny):
        for i in range(nx):
            xi[counter] = np.array([z[k], y[j], x[i]])
            counter += 1
            

In [81]:
interp_vals = interpn(points, actin_field, xi, method='linear')

In [82]:
interp_field = np.zeros((nz, ny, nx))
counter = 0
for k in range(nz):
    for j in range(ny):
        for i in range(nx):
            interp_field[k, j, i] = np.clip(interp_vals[counter], a_min=0, a_max=np.inf)
            counter += 1

In [83]:
viewer = napari.view_image(interp_field)

In [70]:
np.sum(interp_field > 0)

3488

In [63]:
points

(array([0.        , 0.14285714, 0.28571429, 0.42857143, 0.57142857,
        0.71428571, 0.85714286, 1.        ]),
 array([0.        , 0.03448276, 0.06896552, 0.10344828, 0.13793103,
        0.17241379, 0.20689655, 0.24137931, 0.27586207, 0.31034483,
        0.34482759, 0.37931034, 0.4137931 , 0.44827586, 0.48275862,
        0.51724138, 0.55172414, 0.5862069 , 0.62068966, 0.65517241,
        0.68965517, 0.72413793, 0.75862069, 0.79310345, 0.82758621,
        0.86206897, 0.89655172, 0.93103448, 0.96551724, 1.        ]),
 array([0.        , 0.07142857, 0.14285714, 0.21428571, 0.28571429,
        0.35714286, 0.42857143, 0.5       , 0.57142857, 0.64285714,
        0.71428571, 0.78571429, 0.85714286, 0.92857143, 1.        ]))

In [75]:
viewer = napari.view_image(actin_field)